## CS 210 Spring 2024 - Apr 25
### Relational Databases Continued

---

### <font color="brown">Movies Database</font>

The movies data used in this exercise has been sourced from: https://data.world/jamesgaskin/movies/workspace/file?filename=view

This dataset (see **movies_full.csv**) has a bunch of columns, of which we are going to only use title (Column 2), release_date (Column 6), and genre (Column 7). These columns are extracted into a tab-delimited text file, **movies.txt**. There are 615 movies instances in all.

For our movies database, we're going to create a **movies** table, a **user** table, and a **rating** table.

---

#### <font color="brown">Create a Movies Database</font>

##### **1. Create a database named <tt>movies</tt>, and use it**

<pre>
venugopa@data8:~$ mysql
Welcome to the MariaDB monitor.  Commands end with ; or \g.
...

MariaDB [(none)]> create database venugopa_movies;
Query OK, 1 row affected (0.001 sec)

MariaDB [(none)]> use venugopa_movies;
Database changed
</pre>

##### **2. Create the 'movie' table**

<pre>
MariaDB [venugopa_movies]> create table movie (
    -> id int auto_increment primary key,
    -> title varchar(100) not null,
    -> release_date date not null,
    -> genre varchar(20) not null);
</pre>

- The <tt>date</tt> type is for year,month,date but not time: https://dev.mysql.com/doc/refman/8.0/en/datetime.html

**We could make this table more robust by defining the (title,release_date) combination to be unique:**
<pre>
MariaDB [venugopa_movies]> alter table movie add unique(title,release_date);

MariaDB [venugopa_movies]> show create table movie\G;
*************************** 1. row ***************************
       Table: movie
Create Table: CREATE TABLE `movie` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `title` varchar(100) NOT NULL,
  `release_date` date NOT NULL,
  `genre` varchar(20) NOT NULL,
  PRIMARY KEY (`id`),
  UNIQUE KEY `title` (`title`,`release_date`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci
1 row in set (0.00 sec)
</pre>

**You can add the same title multiple times, but the (title,release_date) may not be repeated:**
<pre>
MariaDB [venugopa_movies]> insert into movie (title,release_date,genre) values ('a_movie','2009-12-10','Action');
MariaDB [venugopa_movies]> insert into movie (title,release_date,genre) values ('a_movie','2010-12-10','Adventure');
MariaDB [venugopa_movies]> select * from movie;
+----+---------+--------------+-----------+
| id | title   | release_date | genre     |
+----+---------+--------------+-----------+
|  1 | a_movie | 2009-12-10   | Action    |
|  2 | a_movie | 2010-12-10   | Adventure |
+----+---------+--------------+-----------+
<font color="red">
MariaDB [venugopa_movies]> insert into movie (title,release_date,genre) values ('a_movie','2009-12-10','Drama');
ERROR 1062 (23000): Duplicate entry 'a_movie-2009-12-10' for key 'title'
</font>
</pre>
**Let's drop the movies table and recreate it so that we are can start with id=1 for insertions**
<pre>
MariaDB [venugopa_movies]> drop table movie;
MariaDB [venugopa_movies]> create table movie (
    -> id int auto_increment primary key,
    -> title varchar(100) not null,
    -> release_date date not null,
    -> genre varchar(20) not null);
    -> unique (title,release_date));
</pre>

##### **3. Create the 'user' table**

In [1]:
states = ["Alabama",
        "Alaska",
        "Arizona",
        "Arkansas",
        "California",
        "Colorado",
        "Connecticut",
        "Delaware",
        "Florida",
        "Georgia",
        "Hawaii",
        "Idaho",
        "Illinois",
        "Indiana",
        "Iowa",
        "Kansas",
        "Kentucky",
        "Louisiana",
        "Maine",
        "Maryland",
        "Massachusetts",
        "Michigan",
        "Minnesota",
        "Mississippi",
        "Missouri",
        "Montana",
        "Nebraska",
        "Nevada",
        "New Hampshire",
        "New Jersey",
        "New Mexico",
        "New York",
        "North Carolina",
        "North Dakota",
        "Ohio",
        "Oklahoma",
        "Oregon",
        "Pennsylvania",
        "Rhode Island",
        "South Carolina",
        "South Dakota",
        "Tennessee",
        "Texas",
        "Utah",
        "Vermont",
        "Virginia",
        "Washington",
        "West Virginia",
        "Wisconsin",
        "Wyoming"]

In [2]:
len(states)

50

In [3]:
max([len(s) for s in states])  # this informs the length of the state column in user table

14

<pre>
MariaDB [venugopa_movies]> create table user (
    -> id int auto_increment primary key,
    -> uname char(8) unique not null,
    -> age tinyint not null,
    -> state varchar(14) not null);
</pre>
- The <tt>uname</tt> column values are specified to be unique

##### **4. Create the 'rating' table**

<pre>
MariaDB [venugopa_movies]> create table rating (
    -> userid int not null,
    -> foreign key (userid) references user(id),
    -> movieid int not null,
    -> foreign key (movieid) references movie(id),
    -> unique (userid,movieid),
    -> rating tinyint not null check (rating >=0 and rating <= 10));         
</pre>
- The combination of <tt>userid</tt> and <tt>movieid</tt> values is unique, so a user can only rate a movie once.
- <tt>check</tt> constraints the values of a column:
https://dev.mysql.com/doc/refman/8.0/en/create-table-check-constraints.html

<pre>
MariaDB [venugopa_movies]> show create table rating\G;
*************************** 1. row ***************************
       Table: rating
Create Table: CREATE TABLE `rating` (
  `userid` int(11) NOT NULL,
  `movieid` int(11) NOT NULL,
  `rating` tinyint(4) NOT NULL CHECK (`rating` >= 0 and `rating` <= 10),
  UNIQUE KEY `userid` (`userid`,`movieid`),
  KEY `movieid` (`movieid`),
  CONSTRAINT `rating_ibfk_1` FOREIGN KEY (`userid`) REFERENCES `user` (`id`),
  CONSTRAINT `rating_ibfk_2` FOREIGN KEY (`movieid`) REFERENCES `movie` (`id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci

<font color="red">
MariaDB [venugopa_movies]> insert into rating values (1,1,15);
ERROR 4025 (23000): CONSTRAINT `rating.rating` failed for `venugopa_movies`.`rating`
</font>
</pre>

---

#### <font color="brown">Connect to movies database</font>

In [4]:
# import connector modules
from mysql.connector import connect, Error

In [6]:
# connect to nobels database
try:
    mydb = connect(unix_socket='/run/mysqld/mysqld.sock', database="venugopa_movies")
    cursor = mydb.cursor()
except Error as e:
    print(e)

---

#### <font color="brown">Populate the tables</font>

##### **1. Insert into movie table from movies.txt**

In [8]:
add_movie = "insert into movie (title,release_date,genre) values (%s,%s,%s)"

In [9]:
# read text file and strip the double quotes (if any) from the title
# also, convert date from month/date/year to year/month/date for db date type
import csv
with open("movies.txt") as infile:
    reader = csv.reader(infile,delimiter='\t')
    for row in reader:
        title = row[0].strip('"')  # a few of the movie titles have double quotes around them
        date_parts = row[1].split('/')
        release_date = date_parts[2] + '/' + date_parts[0] + '/' + date_parts[1] # need year/month/day
        genre = row[2]
        cursor.execute(add_movie,(title,release_date,genre))
        mydb.commit()

##### **The movies_db.txt file has 615 records**

In [12]:
def doquery(query):
    cursor.execute(query)
    res = cursor.fetchall()
    for row in res:
        print(row)

In [14]:
doquery('select count(*) from movie')

(615,)


In [15]:
doquery('select * from movie limit 5')

(1, "Look Who's Talking", datetime.date(1989, 10, 12), 'Romance')
(2, 'Driving Miss Daisy', datetime.date(1989, 12, 13), 'Comedy')
(3, 'Turner & Hooch', datetime.date(1989, 7, 28), 'Crime')
(4, 'Born on the Fourth of July', datetime.date(1989, 12, 20), 'War')
(5, 'Field of Dreams', datetime.date(1989, 4, 21), 'Drama')


In [16]:
# getting year, month, and day fields from release date
# look up datetime module and datetime.date class in Python reference
cursor.execute('select * from movie limit 5')
res = cursor.fetchall()
for row in res:
    (id,title,rdate,genre) = row
    rdate = str(rdate.year) + '/' + str(rdate.month) + '/' + str(rdate.day)
    print((id,title,rdate,genre))

(1, "Look Who's Talking", '1989/10/12', 'Romance')
(2, 'Driving Miss Daisy', '1989/12/13', 'Comedy')
(3, 'Turner & Hooch', '1989/7/28', 'Crime')
(4, 'Born on the Fourth of July', '1989/12/20', 'War')
(5, 'Field of Dreams', '1989/4/21', 'Drama')


---

##### **2. Insert into user table**

&lt;uname> &lt;age> &lt;state>

- The uname column is a randomly generated username with 5 letters and 3 digits
- Age is randomly generated, between 18 and 70
- State is one of the US states, randomly chosen

In [17]:
import string, random, numpy as np
letters = [l for l in string.ascii_letters]  # all lowercase letters, followed by all uppercase letters

In [18]:
letters

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [19]:
# generate username, age, and state
def make_user():
    part1 = ''.join(np.random.choice(letters,5,replace=False).tolist())
    part2 = np.random.randint(0,10,3).tolist()
    part2 = ''.join([str(d) for d in part2])
    uname = part1+part2
    age = random.randint(18,71)
    state = random.choice(states)
    return (uname,age,state)

In [21]:
make_user()

('FYwZA627', 41, 'Oregon')

In [22]:
add_user = "insert into user (uname,age,state) values (%s,%s,%s)"

In [23]:
# insert 2000 rows in user table with unique uname
uset = set()  # make sure there are no duplicate usernames
i = 0
while i < 2000:
    user = make_user()
    if not user[0] in uset:
        uset.add(user)
        cursor.execute(add_user,user)
        mydb.commit()
        i += 1

In [24]:
doquery('select count(*) from user')

(2000,)


In [25]:
doquery('select * from user limit 5')

(1, 'nXrTd861', 23, 'Vermont')
(2, 'AzMOD643', 48, 'Maine')
(3, 'DYwcx803', 60, 'Connecticut')
(4, 'aHbYo770', 69, 'Colorado')
(5, 'tXflw292', 24, 'Nebraska')


**Note, your result will be different since user names are randomized**

---

##### **3. Insert into rating table**

Randomly generate a userid (between 1 and 2000, for id values in user table), a movieid (between 1 and 615, for id values in movie table), and a rating (integer between 1 and 10). 


In [26]:
add_rating = "insert into rating (userid,movieid,rating) values (%s,%s,%s)"

**Insert 100000 ratings. This will take a little time, so make sure the following cell finishes executing before you check the database**

In [27]:
# insert 100000 ratings 

import random

ratings = {}  # empty dictionary for mapping (userid,movieid) -> rating 
i = 0
while i < 100000:
    userid = random.randint(1,2000)
    movieid = random.randint(1,615)
    if (userid,movieid) in ratings: # user has already rated this movie
        continue
    rating = random.randint(1,10)
    ratings[(userid,movieid)] = rating
    cursor.execute(add_rating,(userid,movieid,rating))
    mydb.commit()
    i += 1

In [28]:
doquery('select count(*) from rating')

(100000,)


In [29]:
doquery('select * from rating limit 5')

(1, 24, 2)
(1, 32, 8)
(1, 48, 6)
(1, 49, 7)
(1, 54, 6)


**Note, your result will be different since ratings are randomized**

In [30]:
cursor.close()
mydb.close()

---

#### <font color="brown">Queries</font>

##### **The schema**

<pre>
mysql> desc movie;
+--------------+--------------+------+-----+---------+----------------+
| Field        | Type         | Null | Key | Default | Extra          |
+--------------+--------------+------+-----+---------+----------------+
| id           | int          | NO   | PRI | NULL    | auto_increment |
| title        | varchar(100) | NO   | MUL | NULL    |                |
| release_date | date         | NO   |     | NULL    |                |
| genre        | varchar(20)  | NO   |     | NULL    |                |
+--------------+--------------+------+-----+---------+----------------+
4 rows in set (0.00 sec)

mysql> desc user;
+-------+-------------+------+-----+---------+----------------+
| Field | Type        | Null | Key | Default | Extra          |
+-------+-------------+------+-----+---------+----------------+
| id    | int         | NO   | PRI | NULL    | auto_increment |
| uname | char(8)     | NO   | UNI | NULL    |                |
| age   | tinyint     | NO   |     | NULL    |                |
| state | varchar(14) | NO   |     | NULL    |                |
+-------+-------------+------+-----+---------+----------------+
4 rows in set (0.00 sec)

mysql> desc rating;
+---------+---------+------+-----+---------+-------+
| Field   | Type    | Null | Key | Default | Extra |
+---------+---------+------+-----+---------+-------+
| userid  | int     | NO   | PRI | NULL    |       |
| movieid | int     | NO   | PRI | NULL    |       |
| rating  | tinyint | NO   |     | NULL    |       |
+---------+---------+------+-----+---------+-------+
3 rows in set (0.00 sec)

</pre>
**Your results will be different for queries that involve usernames or ratings since these were randomly generated. <p>
**If you want to match results with the ones shown here, you must use the notes_movies.sql database in shared/lectures**

**<font color="brown">1. Find movie titles that begin with 'The', limit to 10</font>**

<pre>
MariaDB [venugopa_movies]> select title from movie where title like 'The%' limit 10;

+--------------------------+
| title                    |
+--------------------------+
| The 40 Year Old Virgin   |
| The Addams Family        |
| The Amazing Spider-Man   |
| The Amazing Spider-Man 2 |
| The Avengers             |
| The Birdcage             |
| The Blair Witch Project  |
| The Blind Side           |
| The Bodyguard            |
| The Boss Baby            |
+--------------------------+
</pre>

---

**<font color="brown">2. How many movies were released in 2015?**

<pre>
MariaDB [venugopa_movies]> select count(*) from movie where release_date like '2015%';

+----------+
| count(*) |
+----------+
|       20 |
+----------+
</pre>
Alternatively:
<pre>
MariaDB [venugopa_movies]> select count(*) from movie where year(release_date) = 2015;

+----------+
| count(*) |
+----------+
|       20 |
+----------+
</pre>
https://dev.mysql.com/doc/refman/8.0/en/date-and-time-functions.html

---

**<font color="brown">3. How many movies were released in the years 2001 to 2005?**

<pre>
MariaDB [venugopa_movies]> select count(*) from movie where year(release_date) between 2001 and 2005;

+----------+
| count(*) |
+----------+
|      100 |
+----------+
</pre>

---

**<font color="brown">4. What are all the movie genres?**

<pre>
MariaDB [venugopa_movies]> select distinct(genre) from movie;

+-----------------+
| genre           |
+-----------------+
| Romance         |
| Comedy          |
| Crime           |
| War             |
| Drama           |
| Family          |
| Action          |
| Animation       |
| Science Fiction |
| Adventure       |
| Thriller        |
| Western         |
| Horror          |
| Mystery         |
| History         |
| Fantasy         |
+-----------------+
</pre>

---

**<font color="brown">5. Which 3 genres had the most releases in 2010?**

<pre>
MariaDB [venugopa_movies]> select genre, count(*) as 'number of releases' 
    ->        from movie 
    ->        where year(release_date) = 2010 
    ->        group by genre 
    ->        order by count(*) desc limit 3;
       
+-----------+--------------------+
| genre     | number of releases |
+-----------+--------------------+
| Animation |                  6 |
| Action    |                  3 |
| Drama     |                  2 |
+-----------+--------------------+
</pre>

Tie for the third entry is broken arbitrarily

---

**<font color="brown">6. List all genres that had at least 3 releases in 2010**

<pre>
MariaDB [venugopa_movies]> select genre, count(*) as 'number of releases' 
    ->          from movie 
    ->          where year(release_date) = 2010 
    ->          group by genre 
    ->          having count(*) > 2;
         
+-----------+-----+
| genre     | num |
+-----------+-----+
| Action    |   3 |
| Animation |   6 |
+-----------+-----+
</pre>


---

**<font color="brown">7. What were the 5 highest rated movies of 2017, and what were their average rating?**

<pre>
MariaDB [venugopa_movies]> select title, round(avg(rating),1) as average_rating 
    ->          from rating, movie 
    ->          where movieid=id and year(release_date) = 2017 
    ->          group by movieid 
    ->          order by average_rating desc 
    ->          limit 5;
         
+------------------------+----------------+
| title                  | average_rating |
+------------------------+----------------+
| Beauty and the Beast   |            5.8 |
| Dunkirk                |            5.8 |
| The LEGO Batman Movie  |            5.7 |
| It                     |            5.6 |
| Spider-Man: Homecoming |            5.6 |
+------------------------+----------------+
</pre>
See https://dev.mysql.com/doc/refman/8.0/en/aggregate-functions.html for various aggregate functions (aside from avg)<br>
See https://dev.mysql.com/doc/refman/8.0/en/mathematical-functions.html for various mathematical functions (aside from round)

---

**<font color="brown">8. Which users have an average rating of at least 8.0 for movies in the years 2010-2015, and what is their average rating</font>**

<pre>
MariaDB [venugopa_movies]> select uname, round(avg(rating),1) as average_rating 
    ->          from rating, user, movie 
    ->          where year(release_date) between 2010 and 2015 and
    ->                userid=user.id and 
    ->                movieid=movie.id
    ->          group by uname 
    ->          having average_rating >= 8.0;
         
+----------+----------------+
| uname    | average_rating |
+----------+----------------+
| bdZWY381 |            8.2 |
| DQtdi052 |            8.0 |
| ePuFV640 |            8.0 |
| haBkx803 |            8.1 |
| QSJUb581 |            8.2 |
| TdMcx810 |            8.0 |
| tvdUE823 |            8.3 |
| vEraH377 |            8.1 |
| VLUPZ650 |            8.3 |
| vqXGj062 |            8.0 |
| yVIBT711 |            8.4 |
+----------+----------------+
</pre>

**Just out of curiosity, what ratings did yVIBT711 give to these movies?**

<pre>
MariaDB [venugopa_movies]> select title,rating 
    ->          from movie,rating,user 
    ->          where uname='yVIBT711' and 
    ->                year(release_date) between 2010 and 2015 and
    ->                userid=user.id and 
    ->                movieid=movie.id;

+-------------------------------------------+--------+
| title                                     | rating |
+-------------------------------------------+--------+
| The Twilight Saga: Eclipse                |      6 |
| The Twilight Saga: Breaking Dawn - Part 2 |     10 |
| Madagascar 3: Europe's Most Wanted        |     10 |
| The Croods                                |      5 |
| Godzilla                                  |     10 |
| Interstellar                              |      9 |
| The Amazing Spider-Man 2                  |      9 |
+-------------------------------------------+--------+
</pre>

---

**<font color="brown">9. Which movies (list title and average rating) released in the same year as 'Wonder Woman' have an average rating greater than it?</font>**

<pre>
MariaDB [venugopa_movies]> select title, avg(rating) as average_rating from movie,user,rating
    -> where year(release_date) in (select year(release_date) from movie where title='Wonder Woman')
    -> and userid=user.id and movieid=movie.id 
    -> group by title
    -> having average_rating > 
    -> (select avg(rating) from movie,user,rating where movie.title='Wonder Woman' and userid=user.id 
    -> and movieid=movie.id)
    -> order by average_rating desc;
                      
+--------------------------------------------------+----------------+
| title                                            | average_rating |
+--------------------------------------------------+----------------+
| Beauty and the Beast                             |         5.8391 |
| Dunkirk                                          |         5.8046 |
| The LEGO Batman Movie                            |         5.6605 |
| It                                               |         5.6218 |
| Thor: Ragnarok                                   |         5.5789 |
| Spider-Man: Homecoming                           |         5.5705 |
| Guardians of the Galaxy Vol. 2                   |         5.5263 |
| Logan                                            |         5.5176 |
| Jumanji: Welcome to the Jungle                   |         5.5083 |
| Coco                                             |         5.5034 |
| Justice League                                   |         5.4937 |
| The Greatest Showman                             |         5.4867 |
| Kong: Skull Island                               |         5.4224 |
| Get Out                                          |         5.3899 |
| The Fate of the Furious                          |         5.3580 |
| Despicable Me 3                                  |         5.2746 |
| The Boss Baby                                    |         5.2389 |
| Pirates of the Caribbean: Dead Men Tell No Tales |         5.2357 |
+--------------------------------------------------+----------------+                
</pre>

Check the average rating for Wonder Woman:

<pre>
MariaDB [venugopa_movies]> select avg(rating) 
    ->     from movie, user, rating
    ->     where movie.title = 'Wonder Woman' and
    ->     userid = user.id and
    ->     movieid = movie.id;
    
+-------------+
| avg(rating) |
+-------------+
|      5.1572 |
+-------------+   
</pre>